# 4.1 将LLM 接入 LangChain

LangChain 为基于 LLM 开发自定义应用提供了高效的开发框架，便于开发者迅速地激发 LLM 的强大能力，搭建 LLM 应用。LangChain 也同样支持多种大模型，内置了 OpenAI、LLAMA 等大模型的调用接口。但是，LangChain 并没有内置所有大模型，它通过允许用户自定义 LLM 类型，来提供强大的可扩展性。

## 1. 基于 LangChain 调用 ChatGPT

LangChain 提供了对于多种大模型的封装，基于 LangChain 的接口可以便捷地调用 ChatGPT 并将其集合在以 LangChain 为基础框架搭建的个人应用中。我们在此简述如何使用 LangChain 接口来调用 ChatGPT。

注意，基于 LangChain 接口调用 ChatGPT 同样需要配置你的个人密钥，配置方法同上。

### 1.1 Models（模型）
从 `langchain.chat_models` 导入 `OpenAI` 的对话模型 `ChatOpenAI` 。 除去OpenAI以外，`langchain.chat_models` 还集成了其他对话模型，更多细节可以查看[Langchain官方文档](https://api.python.langchain.com/en/latest/langchain_api_reference.html#module-langchain.chat_models)。

In [1]:
import os
from dotenv import load_dotenv, find_dotenv

# 读取本地/项目的环境变量。

# find_dotenv()寻找并定位.env文件的路径
# load_dotenv()读取该.env文件，并将其中的环境变量加载到当前的运行环境中  
# 如果你设置的是全局的环境变量，这行代码则没有任何作用。
_ = load_dotenv(find_dotenv())

# 获取环境变量 OPENAI_API_KEY
openai_api_key = os.environ['OPENAI_API_KEY']

没有安装 langchain-openai 的话，请先运行下面进行代码！

In [2]:
from langchain_openai import ChatOpenAI

接下来你需要实例化一个 ChatOpenAI 类，可以在实例化时传入超参数来控制回答，例如 `temperature` 参数。

In [3]:
# 这里我们将参数temperature设置为0.0，从而减少生成答案的随机性。
# 如果你想要每次得到不一样的有新意的答案，可以尝试调整该参数。
llm = ChatOpenAI(temperature=0.0)
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x117efa8f0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x117f14e20>, root_client=<openai.OpenAI object at 0x1157c7d30>, root_async_client=<openai.AsyncOpenAI object at 0x117efa950>, temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********'))

上面的 cell 假设你的 OpenAI API 密钥是在环境变量中设置的，如果您希望手动指定API密钥，请使用以下代码：

In [4]:
# llm = ChatOpenAI(temperature=0, openai_api_key="YOUR_API_KEY")

可以看到，默认调用的是 ChatGPT-3.5 模型。另外，几种常用的超参数设置包括：

    · model_name：所要使用的模型，默认为 ‘gpt-3.5-turbo’，参数设置与 OpenAI 原生接口参数设置一致。

    · temperature：温度系数，取值同原生接口。

    · openai_api_key：OpenAI API key，如果不使用环境变量设置 API Key，也可以在实例化时设置。

    · openai_proxy：设置代理，如果不使用环境变量设置代理，也可以在实例化时设置。

    · streaming：是否使用流式传输，即逐字输出模型回答，默认为 False，此处不赘述。

    · max_tokens：模型输出的最大 token 数，意义及取值同上。

当我们初始化了你选择的`LLM`后，我们就可以尝试使用它！让我们问一下“请你自我介绍一下自己！”

In [5]:
output = llm.invoke("请你自我介绍一下自己！")

In [6]:
output

AIMessage(content='你好，我是一个智能助手，专门为您提供各种服务和帮助。我可以回答您的问题，提供信息和建议，帮助您解决问题。如果您有任何需要，请随时告诉我，我会尽力帮助您。感谢您选择我作为您的助手！如果您有任何问题，请随时问我。', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 109, 'prompt_tokens': 20, 'total_tokens': 129, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c611b32a-4adf-47af-9b97-6dda68a117e1-0', usage_metadata={'input_tokens': 20, 'output_tokens': 109, 'total_tokens': 129})

### 1.2 Prompt (提示模版)

在我们开发大模型应用时，大多数情况下不会直接将用户的输入直接传递给 LLM。通常，他们会将用户输入添加到一个较大的文本中，称为`提示模板`，该文本提供有关当前特定任务的附加上下文。
PromptTemplates 正是帮助解决这个问题！它们捆绑了从用户输入到完全格式化的提示的所有逻辑。这可以非常简单地开始 - 例如，生成上述字符串的提示就是：


我们需要先构造一个个性化 Template：

In [7]:
# 这里我们要求模型对给定文本进行中文翻译
prompt = """请你将由三个反引号分割的文本翻译成英文！\
text: ```{text}```
"""

接下来让我们看一下构造好的完整的提示模版：

In [8]:
text = "我带着比身体重的行李，\
游入尼罗河底，\
经过几道闪电 看到一堆光圈，\
不确定是不是这里。\
"
prompt.format(text=text)

'请你将由三个反引号分割的文本翻译成英文！text: ```我带着比身体重的行李，游入尼罗河底，经过几道闪电 看到一堆光圈，不确定是不是这里。```\n'

我们知道聊天模型的接口是基于消息（message），而不是原始的文本。PromptTemplates 也可以用于产生消息列表，在这种样例中，`prompt`不仅包含了输入内容信息，也包含了每条`message`的信息(角色、在列表中的位置等)。通常情况下，一个 `ChatPromptTemplate` 是一个 `ChatMessageTemplate` 的列表。每个 `ChatMessageTemplate` 包含格式化该聊天消息的说明（其角色以及内容）。

让我们一起看一个示例：

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

template = "你是一个翻译助手，可以帮助我将 {input_language} 翻译成 {output_language}."
human_template = "{text}"

chat_prompt = ChatPromptTemplate([
    ("system", template),
    ("human", human_template),
])

text = "我带着比身体重的行李，\
游入尼罗河底，\
经过几道闪电 看到一堆光圈，\
不确定是不是这里。\
"
messages  = chat_prompt.invoke({"input_language": "中文", "output_language": "英文", "text": text})
messages

[SystemMessage(content='你是一个翻译助手，可以帮助我将 中文 翻译成 英文.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='我带着比身体重的行李，游入尼罗河底，经过几道闪电 看到一堆光圈，不确定是不是这里。', additional_kwargs={}, response_metadata={})]

接下来让我们调用定义好的`llm`和`messages`来输出回答：

In [10]:
output  = llm.invoke(messages)
output

AIMessage(content='I carried luggage heavier than my body and dived into the bottom of the Nile River. After passing through several flashes of lightning, I saw a pile of halos, not sure if this is the place.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 95, 'total_tokens': 138, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c58ae91f-7c0a-4f20-ad7f-8d6421f3a9aa-0', usage_metadata={'input_tokens': 95, 'output_tokens': 43, 'total_tokens': 138})

### 1.3 Output parser（输出解析器）

OutputParsers 将语言模型的原始输出转换为可以在下游使用的格式。 OutputParsers 有几种主要类型，包括：
- 将 LLM 文本转换为结构化信息（例如 JSON） 
- 将 ChatMessage 转换为字符串 
- 将除消息之外的调用返回的额外信息（如 OpenAI 函数调用）转换为字符串

最后，我们将模型输出传递给 `output_parser`，它是一个 `BaseOutputParser`，这意味着它接受**字符串或 BaseMessage 作为输入**。 StrOutputParser 特别简单地将任何输入转换为字符串。

In [11]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
output_parser.invoke(output)

'I carried luggage heavier than my body and dived into the bottom of the Nile River. After passing through several flashes of lightning, I saw a pile of halos, not sure if this is the place.'

从上面结果可以看到，我们通过输出解析器成功将 `ChatMessage` 类型的输出解析为了`字符串`

### 1.4 完整的流程

我们现在可以将所有这些组合成一条链。该链将获取输入变量，将这些变量传递给提示模板以创建提示，将提示传递给语言模型，然后通过（可选）输出解析器传递输出。接下来我们将使用LCEL这种语法去快速实现一条链（chain）。让我们看看它的实际效果！

In [12]:
chain = chat_prompt | llm | output_parser
chain.invoke({"input_language":"中文", "output_language":"英文","text": text})


'I carried luggage heavier than my body weight and dived into the bottom of the Nile River. After passing through several flashes of lightning, I saw a pile of halos, not sure if this is the place.'

再测试一个样例：

In [13]:
text = 'I carried luggage heavier than my body and dived into the bottom of the Nile River. After passing through several flashes of lightning, I saw a pile of halos, not sure if this is the place.'
chain.invoke({"input_language": "英文", "output_language": "中文","text": text})

'我扛着比我的身体还重的行李，潜入尼罗河的底部。穿过几道闪电后，我看到一堆光环，不确定这是否就是目的地。'

> 什么是 LCEL ？ 
LCEL（LangChain Expression Language，Langchain的表达式语言），LCEL是一种新的语法，是LangChain工具包的重要补充，他有许多优点，使得我们处理LangChain和代理更加简单方便。

- LCEL提供了异步、批处理和流处理支持，使代码可以快速在不同服务器中移植。
- LCEL拥有后备措施，解决LLM格式输出的问题。
- LCEL增加了LLM的并行性，提高了效率。
- LCEL内置了日志记录，即使代理变得复杂，有助于理解复杂链条和代理的运行情况。

用法示例：

`chain = prompt | model | output_parser`

上面代码中我们使用 LCEL 将不同的组件拼凑成一个链，在此链中，用户输入传递到提示模板，然后提示模板输出传递到模型，然后模型输出传递到输出解析器。| 的符号类似于 Unix 管道运算符，它将不同的组件链接在一起，将一个组件的输出作为下一个组件的输入。


## 2. 使用 LangChain 调用百度文心一言

我们同样可以通过 LangChain 框架来调用百度文心大模型，以将文心模型接入到我们的应用框架中。

In [14]:

from dotenv import find_dotenv, load_dotenv
import os

# 读取本地/项目的环境变量。

# find_dotenv()寻找并定位.env文件的路径
# load_dotenv()读取该.env文件，并将其中的环境变量加载到当前的运行环境中
# 如果你设置的是全局的环境变量，这行代码则没有任何作用。
_ = load_dotenv(find_dotenv())

# 获取环境变量 API_KEY
QIANFAN_AK = os.environ["QIANFAN_AK"]
QIANFAN_SK = os.environ["QIANFAN_SK"]


In [15]:
from langchain_community.llms.baidu_qianfan_endpoint import QianfanLLMEndpoint

llm = QianfanLLMEndpoint(streaming=True)
res = llm("你好，请你自我介绍一下！")
print(res)

[WARNING][2025-03-05 19:41:13.652] redis_rate_limiter.py:21 [t:8258539328]: No redis installed, RedisRateLimiter unavailable. Ignore this warning if you don't need to use qianfan SDK in distribution environment
/var/folders/yd/c4q_f88j1l70g7_jcb6pdnb80000gn/T/ipykernel_51550/1209153611.py:4: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  res = llm("你好，请你自我介绍一下！")
[ERROR][2025-03-05 19:41:13.835] base.py:134 [t:8258539328]: http request url https://qianfan.baidubce.com/wenxinworkshop/service/list failed with http status code 403
error code from baidu: IamSignatureInvalid
error message from baidu: IamSignatureInvalid, cause: Could not find credential.
request headers: {'User-Agent': 'python-requests/2.32.3', 'Accept-Encoding': 'gzip, deflate, zstd', 'Accept': '*/*', 'Connection': 'keep-alive', 'Content-Type': 'application/json', 'Host': 'qianfan.baidubce.com', 'request-source': '

你好！我是一个人工智能语言模型，我的名字是文心一言。我能够与人进行自然语言交互，并提供各种信息和服务。如果你有任何问题或需要帮助，请随时告诉我，我会尽力为你提供帮助。


## 3. 讯飞星火

我们同样可以通过 LangChain 框架来调用讯飞星火大模型，更多信息参考[SparkLLM](https://python.langchain.com/docs/integrations/llms/sparkllm)

我们希望像调用 ChatGPT 那样直接将秘钥存储在 .env 文件中，并将其加载到环境变量，从而隐藏秘钥的具体细节，保证安全性。因此，我们需要在 .env 文件中配置`IFLYTEK_SPARK_APP_ID`、 `IFLYTEK_SPARK_API_KEY` 和 `IFLYTEK_SPARK_API_SECRET`，并使用以下代码加载：

In [16]:
from dotenv import find_dotenv, load_dotenv
import os

# 读取本地/项目的环境变量。

# find_dotenv()寻找并定位.env文件的路径
# load_dotenv()读取该.env文件，并将其中的环境变量加载到当前的运行环境中
# 如果你设置的是全局的环境变量，这行代码则没有任何作用。
_ = load_dotenv(find_dotenv())

# 获取环境变量 API_KEY
IFLYTEK_SPARK_APP_ID = os.environ["IFLYTEK_SPARK_APP_ID"]
IFLYTEK_SPARK_API_KEY = os.environ["IFLYTEK_SPARK_API_KEY"]
IFLYTEK_SPARK_API_SECRET = os.environ["IFLYTEK_SPARK_API_SECRET"]

另外星火各模型对应的`spark_api_url`与`spark_llm_domain`均不相同，可以参考[接口说明](https://www.xfyun.cn/doc/spark/Web.html#_1-%E6%8E%A5%E5%8F%A3%E8%AF%B4%E6%98%8E)选择调用。

In [17]:
from langchain_community.llms.sparkllm import SparkLLM

# Load the model
llm = SparkLLM(
    model='Spark4.0 Ultra',
    app_id=IFLYTEK_SPARK_APP_ID,
    api_key=IFLYTEK_SPARK_API_KEY,
    api_secret=IFLYTEK_SPARK_API_SECRET,
    spark_api_url="wss://spark-api.xf-yun.com/v4.0/chat",
    spark_llm_domain="4.0Ultra"
    )

In [18]:
res = llm.invoke("你好，请你自我介绍一下！")
print(res)

您好，我是科大讯飞研发的认知智能大模型，我的名字叫讯飞星火认知大模型。我可以和人类进行自然交流，解答问题，高效完成各领域认知智能需求。


从而我们可以将星火大模型加入到 LangChain 架构中，实现在应用中对文心大模型的调用。

## 4. 使用 LangChain 调用智谱 GLM

我们同样可以通过 LangChain 框架来调用智谱 AI 大模型，以将其接入到我们的应用框架中。由于 langchain 中提供的[ChatGLM](https://python.langchain.com/docs/integrations/llms/chatglm)已不可用，因此我们需要自定义一个LLM。

如果你使用的是智谱 GLM API，你需要将我们封装的代码[zhipuai_llm.py](./zhipuai_llm.py)下载到本 Notebook 的同级目录下，才可以运行下列代码来在 LangChain 中使用 GLM。

根据智谱官方宣布以下模型即将弃用，在这些模型弃用后，会将它们自动路由至新的模型。请用户注意在弃用日期之前，将您的模型编码更新为最新版本，以确保服务的顺畅过渡，更多模型相关信息请访问[model](https://open.bigmodel.cn/dev/howuse/model)

| 模型编码 |弃用日期|指向模型|
| ---- | ---- | ---- |
|chatglm_pro|2024 年 12 月 31 日|glm-4|
|chatglm_std|2024 年 12 月 31 日|glm-3-turbo|
|chatglm_lite|2024 年 12 月 31 日|glm-3-turbo|

### 4.1 自定义chatglm接入 langchain

In [19]:
from zhipuai_llm import ZhipuaiLLM
from dotenv import find_dotenv, load_dotenv
import os

# 读取本地/项目的环境变量。

# find_dotenv()寻找并定位.env文件的路径
# load_dotenv()读取该.env文件，并将其中的环境变量加载到当前的运行环境中
# 如果你设置的是全局的环境变量，这行代码则没有任何作用。
_ = load_dotenv(find_dotenv())

# 获取环境变量 API_KEY
api_key = os.environ["ZHIPUAI_API_KEY"] #填写控制台中获取的 APIKey 信息

In [20]:
zhipuai_model = ZhipuaiLLM(model_name="glm-4-plus", temperature=0.1, api_key=api_key)

In [21]:
zhipuai_model.invoke("你好，请你自我介绍一下！")

AIMessage(content='你好！我是人工智能助手智谱清言（ChatGLM），是基于智谱 AI 公司于 2024 年训练的语言模型开发的。我的任务是针对用户的问题和要求提供适当的答复和支持。', additional_kwargs={}, response_metadata={'time_in_seconds': 1.87}, id='run-4e509a7e-9859-4acb-9418-23245fa5b7a7-0', usage_metadata={'input_tokens': 11, 'output_tokens': 42, 'total_tokens': 53})